# Benchmark HUC8 Streamflow Aggregation Queries

This notebook show how to query the NWM reanalysis dataset using HUC8s in various ways, time the queries, and save the results in a CSV.

## Setup

In [1]:
import json
from os.path import join
import time

from tqdm.notebook import tqdm
import geopandas as gpd
import xarray as xr
import fsspec
import numpy as np
import pyproj
from dask.distributed import Client
from dask_gateway import Gateway
import dask.dataframe as dd
import numpy as np
import pandas as pd
import fsspec

%matplotlib inline

def get_json(uri):
    with fsspec.open(uri) as fd:
        return json.load(fd)

In [2]:
# Connect to existing cluster using cluster.name

# This constant needs to be set!
cluster_name = ''
gateway = Gateway()
cluster = gateway.connect(cluster_name)
client = cluster.get_client()

In [3]:
# Get COMIDs for a HUC8 around Philly from a HUC8 extract on S3.
# Each COMID represents a stream reach.

# Location of HUC8 extract JSON files.
huc8_root_uri = 's3://azavea-noaa-hydro-data/noaa/huc8-extracts/transformed/'

# TODO: run this with multiple HUC8s.
philly_huc8 = '02040202'
huc8_uri = join(huc8_root_uri, f'{philly_huc8}.json')

huc8_dict = get_json(huc8_uri)
comids = huc8_dict['features'][0]['properties']['comids']

zarr_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/lf/07-07-2022c/nwm-subset.zarr'
ds = xr.open_zarr(zarr_uri)

# Apparently, only some of the reach ids in NHDPlus V2 are available in NWM.
# Question: why is that?
avail_comids = list(set(ds.feature_id.values).intersection(set(comids)))

# Need to sort or you will get warnings about a slowdown with an out of order index.
avail_comids.sort()
print(
    f'There are {len(comids)} reaches in the HUC and {len(avail_comids)} of those are in NWM.')

del ds

There are 2153 reaches in the HUC and 1787 of those are in NWM.


## Run timing experiments

In [24]:
# Set things commons across all experiments.

# The number of times to repeat execution
nb_repeats = 5
time_ranges = [
    slice('1990-01-01', '2000-01-01'), 
    slice('1990-01-01', '1991-01-01'),
    slice('1990-01-01', '1990-02-01'),
]
nb_reaches = len(avail_comids)
nb_workers = len(client.scheduler_info()['workers'])

### Zarr experiments

In [14]:
# Settings

# A map from query nicknames to functions that execute the query.
query_map = {
    'mean_features_mean_day': (lambda ds: ds.streamflow.mean(dim='feature_id').groupby('time.dayofyear').mean().values),
    'mean_day': (lambda ds: ds.streamflow.groupby('time.dayofyear').mean().values),
    'mean_week': (lambda ds: ds.streamflow.groupby('time.weekofyear').mean().values)
}
data_format = 'zarr'

# The CHRTOUT data from the NWM Retrospective Zarr 2.1 dataset
# This has "Streamflow values at points associated with flow lines" 
# See https://registry.opendata.aws/nwm-archive/
# Original dataset is at s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr

# 10 year subset and transposed chunk version generated by save_zarr_data.ipynb
zarr_orig_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/lf/07-07-2022c/nwm-subset.zarr'
zarr_trans_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/lf/07-07-2022c/trans-chunk.zarr'
zarr_uris = [zarr_orig_uri, zarr_trans_uri]

# Set this to a new URI!
zarr_results_uri = ''

In [10]:
# Run this block to use shorter test settings.
query_map = {
    'mean_day': query_map['mean_day']}
zarr_uris = zarr_uris[0:1]
nb_repeats = 3

In [15]:
%%time

zarr_exp_rows = []
for zarr_uri in tqdm(zarr_uris, desc='zarr stores', leave=False):
    ds = xr.open_zarr(zarr_uri)
    for time_range in tqdm(time_ranges, desc='time ranges', leave=False):
        sub_ds = ds.sel(feature_id=avail_comids, time=time_range)

        nb_days = (pd.to_datetime(time_range.stop) - pd.to_datetime(time_range.start)).days
        
        chunk_sizes = np.array([ds.streamflow.chunks[0][0], ds.streamflow.chunks[1][0]])
        time_chunk_sz = chunk_sizes[0]
        feature_id_chunk_sz = chunk_sizes[1]
        
        for qname, qfunc in tqdm(query_map.items(), desc='query', leave=False):
            for repeat_ind in tqdm(range(nb_repeats), desc='repeat', leave=False):
                start_time = time.time()
                vals = qfunc(sub_ds)
                elapsed = time.time() - start_time
                exp_row = {
                    'query': qname,
                    'time': elapsed,
                    'nb_reaches': nb_reaches,
                    'nb_days': nb_days,
                    'data_format': data_format,
                    'time_chunk_sz': time_chunk_sz,
                    'feature_id_chunk_sz': feature_id_chunk_sz, 
                    'nb_workers': nb_workers,
                    'repeat_ind': repeat_ind
                }
                zarr_exp_rows.append(exp_row)
        del sub_ds
    del ds

zarr stores:   0%|          | 0/2 [00:00<?, ?it/s]

time ranges:   0%|          | 0/3 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


query:   0%|          | 0/3 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

...

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalenda

CPU times: user 2min 12s, sys: 4.63 s, total: 2min 17s
Wall time: 12min 9s


In [16]:
exp_rows = zarr_exp_rows
df = pd.DataFrame(exp_rows)
df.to_csv(zarr_results_uri)
df

,query,time,nb_reaches,nb_days,data_format,time_chunk_sz,feature_id_chunk_sz,nb_workers,repeat_ind
0,mean_features_mean_day,25.575601,1787,3652,zarr,672,30000,16,0
1,mean_features_mean_day,24.883382,1787,3652,zarr,672,30000,16,1
2,mean_features_mean_day,24.268302,1787,3652,zarr,672,30000,16,2
3,mean_features_mean_day,23.722426,1787,3652,zarr,672,30000,16,3
4,mean_features_mean_day,24.381311,1787,3652,zarr,672,30000,16,4
...,...,...,...,...,...,...,...,...,...
85,mean_week,1.196906,1787,31,zarr,30000,672,16,0
86,mean_week,1.176692,1787,31,zarr,30000,672,16,1
87,mean_week,1.218968,1787,31,zarr,30000,672,16,2
88,mean_week,1.388866,1787,31,zarr,30000,672,16,3


### Parquet experiments

In [4]:
# Settings

# A map from query nicknames to functions that execute the query.
query_map = {
    'mean_day': (lambda df: df.groupby(df.index.dayofyear).streamflow.mean().values.compute()),
    'mean_week': (lambda df: df.groupby(df.index.weekofyear).streamflow.mean().values.compute()),
    'mean_features_mean_day': (lambda df: df.groupby(['feature_id',df.index.dayofyear]).streamflow.mean().values.compute()),
}
data_format = 'parquet'

# 10 years with metadata
parq_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/parquet/vl/07-13-2022c_metadata/'
parq_uris = [parq_uri]

parq_results_uri = ''

In [8]:
%%time

parq_exp_rows = []
for parq_uri in tqdm(parq_uris, desc='parqet stores', leave=False):
    df = dd.read_parquet(parq_uri, engine='pyarrow', index='time', columns=['feature_id','streamflow'])
    for time_range in tqdm(time_ranges, desc='time ranges', leave=False):
        sub_df = df.query(
            'feature_id in @avail_comids and time > @start_time and time < @end_time', 
            local_dict={
                'avail_comids': avail_comids, 
                'start_time': time_range.start,
                'end_time': time_range.stop})

        nb_days = (pd.to_datetime(time_range.stop) - pd.to_datetime(time_range.start)).days
        
        # TODO: make the next lines of code valid
        time_chunk_sz = -9999
        feature_id_chunk_sz = -9999
        
        for qname, qfunc in tqdm(query_map.items(), desc='query', leave=False):
            for repeat_ind in tqdm(range(nb_repeats), desc='repeat', leave=False):
                start_time = time.time()
                vals = qfunc(sub_df)
                elapsed = time.time() - start_time
                exp_row = {
                    'query': qname,
                    'time': elapsed,
                    'nb_reaches': nb_reaches,
                    'nb_days': nb_days,
                    'data_format': data_format,
                    'time_chunk_sz': time_chunk_sz,
                    'feature_id_chunk_sz': feature_id_chunk_sz,
                    'repeat_ind': repeat_ind
                }
                parq_exp_rows.append(exp_row)
        del sub_df
    del df

parqet stores:   0%|          | 0/1 [00:00<?, ?it/s]

time ranges:   0%|          | 0/3 [00:00<?, ?it/s]

query:   0%|          | 0/3 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

...

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/dataframe/accessor.py:84: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  out = getattr(getattr(obj, accessor, obj), attr)
/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/dataframe/accessor.py:84: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  out = getattr(getattr(obj, accessor, obj), attr)
/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/dataframe/accessor.py:84: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which retu

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 1min 26s, sys: 1.53 s, total: 1min 28s
Wall time: 1h 1min 6s


In [ ]:
## df = pd.DataFrame(parq_exp_rows)
## df.to_csv(parq_results_uri)
## df

In [14]:
parq_results_uri = 's3://azavea-noaa-hydro-data/esip-experiments/benchmarks/vl/07-20-2022c.csv'
results = pd.DataFrame(parq_exp_rows)
results.to_csv(parq_results_uri)
results

,query,time,nb_reaches,nb_days,data_format,time_chunk_sz,feature_id_chunk_sz,repeat_ind
0,mean_day,88.281882,1787,31,parquet,-9999,-9999,0
1,mean_day,77.788391,1787,31,parquet,-9999,-9999,1
2,mean_day,76.687478,1787,31,parquet,-9999,-9999,2
3,mean_day,74.802847,1787,31,parquet,-9999,-9999,3
4,mean_day,75.135556,1787,31,parquet,-9999,-9999,4
5,mean_week,74.464998,1787,31,parquet,-9999,-9999,0
6,mean_week,73.857460,1787,31,parquet,-9999,-9999,1
7,mean_week,74.479352,1787,31,parquet,-9999,-9999,2
8,mean_week,73.761637,1787,31,parquet,-9999,-9999,3
9,mean_week,73.977649,1787,31,parquet,-9999,-9999,4
